In [1]:
# Import libraries

import sys
import logging
import pandas as pd
import xarray as xr
from pathlib import Path

root_path = Path(globals()['_dh'][0]).resolve().parent
sys.path.append(str(root_path))

import paths
from library.assumptions import read_assumptions
from library.weather import load_weather
from library.network import build_network

logging.basicConfig(level=logging.INFO)

In [3]:
# Set the configuration

## Parameters you won't change very often
base_currency = 'SEK'
exchange_rates = {
    'EUR': 11.68,
    'USD': 10.70
}
base_year = 2024
discount_rate = 0.05
onwind_turbine =  "2030_5MW_onshore.yaml"
offwind_turbine = "2030_20MW_offshore.yaml"
resolution = 3

## Parameters that will change frequently
target_year = 2030
use_offwind = False
use_h2 = False
h2_initial = 0
biogas_limit = 0
load_target = 10 #TWh

In [4]:
# Load the data needed from assumptions, the electricity demand, and the atlite output from ERA5 weather data for VGR 2023

## Transform assumptions to range base_year to target_year
assumptions = read_assumptions(paths.input_path / 'assumptions.csv', base_year, target_year, base_currency, exchange_rates, discount_rate)

# Read the normalized demand from csv file (see normalize_demand() in library.demand for details)
# And then calculate target_load using projection of energy need in target_year
normalized_demand = pd.read_csv(paths.input_path / 'demand/normalized-demand-2023-3h.csv', delimiter=',')
target_load = load_target * normalized_demand['value'].values.flatten() * 1_000_000

# Create of load the cutout from atlite (we assume weather data from 2023 and a 3h window)
geo = '14' # All of VGR
section = None
cutout, selection, index = load_weather(geo, section, '2023-01', '2023-12')
geography = selection.total_bounds  

capacity_factor_solar = xr.open_dataarray(paths.input_path / 'renewables' / f"capacity-factor-{geo}-2023-01-2023-12-solar.nc").values.flatten()
capacity_factor_onwind = xr.open_dataarray(paths.input_path / 'renewables' / f"capacity-factor-{geo}-2023-01-2023-12-onwind.nc").values.flatten()
capacity_factor_offwind = xr.open_dataarray(paths.input_path / 'renewables' / f"capacity-factor-{geo}-2023-01-2023-12-offwind.nc").values.flatten()

In [5]:
# Build the network

network = build_network(index, resolution, geography, target_load, assumptions, capacity_factor_solar, capacity_factor_onwind, capacity_factor_offwind, use_offwind, use_h2, h2_initial, biogas_limit)

In [6]:
# Add constraints to the model and run the optimization

## Create the model
model = network.optimize.create_model()

generator_capacity = model.variables["Generator-p_nom"]
link_capacity = model.variables["Link-p_nom"]

## Add offwind constraint
if use_offwind:
    offwind_percentage = 0.5

    offwind_constraint = (1 - offwind_percentage) / offwind_percentage * generator_capacity.loc['offwind'] - generator_capacity.loc['onwind']
    model.add_constraints(offwind_constraint == 0, name="Offwind_constraint")

## Add battery charge/discharge ratio constraint
lhs = link_capacity.loc["Battery charge"] - network.links.at["Battery charge", "efficiency"] * link_capacity.loc["Battery discharge"]
model.add_constraints(lhs == 0, name="Link-battery_fix_ratio")

## Run optimization
network.optimize.solve_model(solver_name='highs')

INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing integer variables.: 100%|██████████| 1/1 [00:00<00:00, 666.71it/s]
INFO:linopy.io: Writing time: 0.28s
INFO:linopy.solvers:Log file at /tmp/highs.log


Running HiGHS 1.7.2 (git hash: 184e327): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [4e-06, 5e+00]
  Cost   [8e+01, 1e+06]
  Bound  [0e+00, 0e+00]
  RHS    [7e+02, 2e+03]
Presolving model
23372 rows, 14616 cols, 54050 nonzeros  0s
17776 rows, 9020 cols, 51605 nonzeros  0s
17306 rows, 8550 cols, 53482 nonzeros  0s

Solving MIP model with:
   17306 rows
   8550 cols (0 binary, 2 integer, 0 implied int., 8548 continuous)
   53482 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   -inf            inf                  inf        0      0      0         0     0.1s
 S       0       0         0   0.00%   -inf            17403308161.15     Large        0      0      0         0     0.6s
 R       0       0         0   0.00

INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 23368 primals, 52569 duals
Objective: 1.74e+10
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-ext-p-lower, Generator-ext-p-upper, Link-fix-p-lower, Link-fix-p-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, Store-energy_balance were not assigned to the network.


Writing the solution to /tmp/linopy-solve-zw6iteff.sol


('ok', 'optimal')

In [18]:
network.buses_t.marginal_price['Load bus']

snapshot
2023-01-01 00:00:00    0.0
2023-01-01 03:00:00    0.0
2023-01-01 06:00:00    0.0
2023-01-01 09:00:00    0.0
2023-01-01 12:00:00    0.0
                      ... 
2023-12-31 09:00:00    0.0
2023-12-31 12:00:00    0.0
2023-12-31 15:00:00    0.0
2023-12-31 18:00:00    0.0
2023-12-31 21:00:00    0.0
Name: Load bus, Length: 2920, dtype: float64

In [19]:
network.statistics()

Optimal Capacity  Installed Capacity        Supply  \
Generator backstop       1686.392061                 0.0  4.455970e+05   
          onwind         4663.362857                 0.0  5.476981e+06   
          solar          4824.257340                 0.0  4.912774e+06   
Link      AC             2622.108901                 0.0  9.965499e+06   
          li-ion         2733.738374                 0.0  9.554403e+06   
Load      -                 0.000000                 0.0  0.000000e+00   
Store     li-ion        16659.511116                 0.0  2.118789e+06   

                      Withdrawal      Dispatch  Transmission  Capacity Factor  \
Generator backstop  1.880575e-10  4.455970e+05           0.0         0.030163   
          onwind    0.000000e+00  5.476981e+06           0.0         0.134072   
          solar     0.000000e+00  4.912774e+06           0.0         0.116250   
Link      AC        1.038975e+07 -4.242552e+05           0.0         0.452325   
          li-ion    9.961157e+06 -4.067539e+05           0.0         0.415957   
Load      -         1.000000e+07 -1.000000e+07           0.0              NaN   
Store     li-ion    2.123131e+06 -4.342512e+03           0.0         0.371980   

                     Curtailment  Capital Expenditure  \
Generator backstop  0.000000e+00         0.000000e+00   
          onwind    1.015839e+07         6.286029e+09   
          solar     5.370525e+05         2.505845e+09   
Link      AC        0.000000e+00         5.595347e+08   
          li-ion    0.000000e+00         0.000000e+00   
Load      -         0.000000e+00         0.000000e+00   
Store     li-ion    0.000000e+00         3.208489e+09   

                    Operational Expenditure  Revenue  Market Value  
Generator backstop             4.684115e+09      0.0           NaN  
          onwind               1.576252e+08      0.0           NaN  
          solar                0.000000e+00      0.0           NaN  
Link      AC                   0.000000e+00      0.0           NaN  
          li-ion               0.000000e+00      0.0           NaN  
Load      -                    0.000000e+00      0.0           0.0  
Store     li-ion              -1.218226e+05      0.0           NaN

In [ ]:
## Format and create the system table widget widget
generator_column_names = {
    'capital_cost': 'Capital Cost',
    'marginal_cost': 'Marginal Cost',
    'lifetime': 'Lifetime',
    'p_nom_opt': 'Optimal Capacity'
}

store_column_names = {
    'capital_cost': 'Capital Cost',
    'marginal_cost': 'Marginal Cost',
    'lifetime': 'Lifetime',
    'e_nom_opt': 'Optimal Capacity'
}

generators = pd.concat([
    network.generators.loc[['Solar park', 'Onwind park', 'Offwind park']][['capital_cost', 'marginal_cost', 'lifetime', 'p_nom_opt']],
    network.links.loc[['Gas turbine']][['capital_cost', 'marginal_cost', 'lifetime', 'p_nom_opt']]
])

generators['capital_cost'] = generators['capital_cost'].apply(lambda x: f"{round(float(x)/1_000_000, 1)} MSEK/MW")
generators['marginal_cost'] = generators['marginal_cost'].apply(lambda x: f"{round(float(x), 1)} SEK/MWh")
generators['lifetime'] = generators['lifetime'].apply(lambda x: f"{int(round(float(x), 0))} years")
generators['p_nom_opt'] = generators['p_nom_opt'].apply(lambda x: f"{int(round(float(x), 0))} MW")
generators.rename(columns=generator_column_names, inplace=True)

stores = network.stores.loc[["H2 storage", "Battery storage"]][['capital_cost', 'marginal_cost', 'lifetime', 'e_nom_opt']]
stores.loc[:, 'capital_cost'] = stores['capital_cost'].apply(lambda x: f"{round(float(x)/1_000_000,1)} MSEK/MWh")
stores.loc[:, 'marginal_cost'] = stores['marginal_cost'].apply(lambda x: f"{round(float(x),1)} SEK/MWh")
stores.loc[:, 'lifetime'] = stores['lifetime'].apply(lambda x: f"{int(round(float(x), 0))} years")
stores.loc[:, 'e_nom_opt'] = stores['e_nom_opt'].apply(lambda x: f"{int(round(float(x), 0))} MWh")
stores.rename(columns=store_column_names, inplace=True)

system_table = widgets.HTML(value=pd.concat([generators, stores]).to_html())


## Format and create the stores table widget widget

## Create generators size pie chart widget
generator_pie_chart = widgets.Output()
with generator_pie_chart:
    network.generators['p_nom_opt'].plot(figsize=(4, 4), kind='pie', labels=None, legend=True, startangle=90, title='Generator sizes')
    plt.show()

## Create stores size pie chart widget
store_pie_chart = widgets.Output()
with store_pie_chart:
    network.stores.loc[["H2 storage", "Battery storage"]]['e_nom_opt'].plot(figsize=(4, 4), kind='pie', labels=None, legend=True, startangle=90, title='Energy store sizes')
    plt.axis('off')  # Turns off the axis
    plt.show()

## Create price line chart widget
price_line_chart = widgets.Output()
with price_line_chart:
    network.buses_t.marginal_price.plot(figsize=(24, 6), legend=True, title='Marginal price')
    plt.show()

## Create generators outpuExclusionContainert line chart widget
generator_line_chart = widgets.Output()
with generator_line_chart:
    fig, ax = plt.subplots(figsize=(16,6))
    network.generators_t.p.abs().plot(ax=ax, kind='area', stacked=True, legend=True, alpha=0.2, title='Power output')
    network.generators_t.p.abs().rolling(window=window_size, center=True).mean().plot(ax=ax, kind='area', stacked=True)
    plt.show()

## Create generators total output pie chart
generator_pie_chart = widgets.Output()
with generator_pie_chart:
    network.generators_t.p.sum().plot(figsize=(8, 6), kind='pie', labels=None, legend=True, title='Power output')
    plt.show()


## Create stores line chart widget
store_line_chart = widgets.Output()
with store_line_chart:
    network.stores_t.e[['H2 storage', 'Battery storage']].plot(figsize=(24, 6),legend=True, title='Energy stored')
    plt.show()


## Build the dashboard and show

sun_wind_map = widgets.HBox([solar_map, onwind_map, offwind_map])
sun_wind_cfs = widgets.VBox([solar_capacity_line_chart, onwind_capacity_line_chart, offwind_capacity_line_chart])

row_1 = widgets.HBox([widgets.VBox([load_line_chart, sun_wind_map]), sun_wind_cfs])

row_2 = widgets.HBox([widgets.VBox([network_map, system_table, widgets.HBox([generator_pie_chart, store_pie_chart])]), parameters_table])

row_3 = widgets.HBox([generator_line_chart, generator_pie_chart])

dashboard = widgets.VBox([row_1, row_2, row_3, store_line_chart, price_line_chart])

dashboard


In [ ]:
bus_colors = ["black", "yellow", "blue", "green", "lightblue", "red", "red", "red", "red", "black"]

minx, miny, maxx, maxy = SELECTION.total_bounds
warnings.filterwarnings("ignore", category=UserWarning, module="cartopy.mpl.feature_artist")
network_map = widgets.Output()
fig, ax = plt.subplots(figsize=(12, 12), subplot_kw={'projection': ccrs.PlateCarree()})
NETWORK.plot(ax=ax, boundaries = [minx-1, maxx+3, miny-1.5, maxy+4.5], color_geomap=True, bus_sizes=0.02, bus_colors=bus_colors)
legend_labels = [plt.Line2D([0], [0], marker='o', color='w', label=bus, 
                        markerfacecolor=color, markersize=10) for bus, color in zip(network.buses.index, bus_colors)]
plt.legend(handles=legend_labels, loc='upper left', bbox_to_anchor=(0.6, 0.7), ncol=1)


In [ ]:
fix, ax1 = plt.subplots(figsize=(24, 6))
                        
NETWORK.buses_t.marginal_price[['Load bus']].plot(ax=ax1, legend=True, title='Marginal price')

ax2 = plt.twinx(ax1)
NETWORK.stores_t.e[['H2 storage']].plot(ax=ax2,legend=True, color='red', title='Energy stored')


In [ ]:
start_date = pd.to_datetime('2023-01-01')
end_date = pd.to_datetime('2023-12-31')

fix, ax1 = plt.subplots(figsize=(24, 6))
                        
NETWORK.buses_t.marginal_price[['Load bus']].loc[start_date:end_date].plot(ax=ax1, legend=True, title='Marginal price')

#ax2 = plt.twinx(ax1)
#DEMAND.plot(ax=ax2, color='grey', alpha=0.2)

#NETWORK.buses_t.marginal_price.loc[start_date:end_date].mean()


In [ ]:
total_cost = NETWORK.statistics.capex().sum()
total_energy = NETWORK.generators_t.p[['Solar park', 'Onwind park', 'Offwind park', 'Biogas market']].sum().sum()

total_cost/total_energy

In [ ]:
network.statistics.capex()

In [ ]:
NETWORK.generators_t.p.abs().rolling(window=56, center=True).mean().plot(figsize=(16,6), kind='area', stacked=True)

In [ ]:
NETWORK.generators_t.p.abs().plot(figsize=(16,6), kind='area', stacked=True)

In [ ]:
fix, ax1 = plt.subplots(figsize=(24, 6))                   
DEMAND.plot(ax=ax1, color='grey', alpha=0.2)

In [ ]:
column_names = {
    'annual_cost': 'Årskostnad',
    'lifetime': 'Livslängd',
    'p_nom_opt': 'Kapacitet',
    'generators': 'Kraftverk',
    'energy_produced': 'Producerad energi',
}

generator_index = NETWORK.generators.index.difference(['Backstop', 'Biogas input'])

generators = pd.concat([
    NETWORK.generators.loc[NETWORK.generators.index.isin(generator_index)][['capital_cost', 'marginal_cost', 'lifetime', 'p_nom_opt', 'p_nom_mod']],
    NETWORK.links.loc[['Combined Cycle Gas turbine', 'Simple Cycle Gas turbine']][['capital_cost', 'marginal_cost', 'lifetime', 'p_nom_opt', 'p_nom_mod']]
])

generators['annual_cost'] = np.where(generators['p_nom_opt'] != 0, generators['capital_cost'] * generators['p_nom_opt'], 0)
generators['energy_produced'] = NETWORK.generators_t.p.loc[:, generator_index].sum() * 3
generators['generators'] = generators['p_nom_opt'] / generators['p_nom_mod']

other = NETWORK.links.loc[['H2 electrolysis', 'Battery charge']][['capital_cost', 'marginal_cost', 'lifetime', 'p_nom_opt']]
other['annual_cost'] = np.where(other['p_nom_opt'] != 0, other['capital_cost'] * other['p_nom_opt'], 0)

stores = NETWORK.stores.loc[["H2 storage", "Battery storage"]][['capital_cost', 'marginal_cost', 'lifetime', 'e_nom_opt']]
stores['annual_cost'] = np.where(stores['e_nom_opt'] != 0, stores['capital_cost'] * stores['e_nom_opt'], 0)
stores.rename(columns={'e_nom_opt': 'p_nom_opt'}, inplace=True)

biogas = NETWORK.generators.loc[['Biogas input']][['capital_cost', 'marginal_cost', 'lifetime', 'p_nom_opt']]
biogas['energy_produced'] = NETWORK.generators_t.p[['Biogas input']].sum() * 3 * float(parameters.loc['biogas', 'efficiency'].value)
biogas['annual_cost'] = np.where(biogas['p_nom_opt'] != 0, biogas['marginal_cost'] * biogas['energy_produced'] / float(parameters.loc['biogas', 'efficiency'].value), 0)
biogas[['lifetime', 'p_nom_opt', 'total_cost']] = 0

system = pd.concat([generators, other, stores, biogas]).drop(columns=['capital_cost', 'marginal_cost'])[['p_nom_opt', 'generators', 'lifetime', 'energy_produced', 'annual_cost']]

totals = pd.DataFrame(columns=system.columns, index=['Total'])
totals['energy_produced'] = system['energy_produced'].sum()
totals['annual_cost'] = system['annual_cost'].sum()

system = pd.concat([system, totals]).rename(columns=column_names)

years = lambda s: f'{s:,.0f} år'.replace(',', ' ')
million_cost = lambda s: f'{s/1_000_000:,.0f} MSEK'.replace(',', ' ')
energy = lambda s: f'{s/1_000:,.0f} GWh'.replace(',', ' ')
price = lambda s: f'{s:,.0f} SEK'.replace(',', ' ')
system.abs().style.format({'Livslängd': years, 'Producerad energi': energy, 'Pris': price, 'Årskostnad': million_cost, 'Totalkostnad': million_cost}, precision=0, na_rep='-')

In [ ]:
f"{system['Årskostnad']['Total'] / (NETWORK.loads_t.p['Desired load'].sum() * 3) / 1000 * 100:.1f} öre/kWh"

In [ ]:
start_date = pd.to_datetime('2023-08-01')
end_date = pd.to_datetime('2023-09-01')
fig1, ax1 = plt.subplots(figsize=(24, 6))
NETWORK.generators_t.p[['Solar park']].loc[start_date:end_date].plot(ax=ax1, kind='line', legend=True, alpha=0.2, title='Power output')
NETWORK.links_t.p0[['Solar battery link', 'Solar H2 link', 'Solar load link']].loc[start_date:end_date].plot(ax=ax1, kind='area', stacked=True, legend=True, alpha=0.2)

fig2, ax2 = plt.subplots(figsize=(24, 6))
NETWORK.generators_t.p[['Onwind park']].loc[start_date:end_date].plot(ax=ax2, kind='line', legend=True, alpha=0.2, title='Power output')
NETWORK.links_t.p0[['Onwind battery link', 'Onwind H2 link', 'Onwind load link']].loc[start_date:end_date].plot(ax=ax2, kind='area', stacked=True, legend=True, alpha=0.2)

fig3, ax3 = plt.subplots(figsize=(24, 6))
NETWORK.generators_t.p[['Offwind park']].loc[start_date:end_date].plot(ax=ax3, kind='line', legend=True, alpha=0.2, title='Power output')
NETWORK.links_t.p0[['Offwind battery link', 'Offwind H2 link', 'Offwind load link']].loc[start_date:end_date].plot(ax=ax3, kind='area', stacked=True, legend=True, alpha=0.2)

In [ ]:
solar_curtailment = NETWORK.generators_t.p_max_pu[['Solar park']] * NETWORK.generators.loc['Solar park']['p_nom_opt'] - NETWORK.generators_t.p[['Solar park']]
solar_curtailment['Curtailment'] = solar_curtailment['Solar park']
solar_curtailment.drop(columns=['Solar park'], inplace=True)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 6))

pd.concat([NETWORK.links_t.p0[['Solar battery link', 'Solar H2 link', 'Solar load link']], solar_curtailment]).resample('ME').sum().plot(ax=ax1, kind='bar', stacked=True, legend=True, alpha=0.2)

wind_curtailment = NETWORK.generators_t.p_max_pu[['Onwind park']] * NETWORK.generators.loc['Onwind park']['p_nom_opt'] - NETWORK.generators_t.p[['Onwind park']]
wind_curtailment['Curtailment'] = wind_curtailment['Onwind park']
wind_curtailment.drop(columns=['Onwind park'], inplace=True)

pd.concat([NETWORK.links_t.p0[['Onwind battery link', 'Onwind H2 link', 'Onwind load link']], wind_curtailment]).resample('ME').sum().plot(ax=ax2, kind='bar', stacked=True, legend=True, alpha=0.2)

In [ ]:
data = pd.concat([NETWORK.links_t.p0[['Solar battery link', 'Solar H2 link', 'Solar load link']], solar_curtailment]).resample('ME').sum()

# Prepare data for Plotly
data = data.reset_index()

# Create an interactive bar chart using Plotly
fig = go.Figure()

for column in data.columns[1:]:  # Skip the first column as it is the index
    fig.add_trace(go.Bar(
        x=data['snapshot'],  # Use the reset index for the x-axis
        y=data[column],
        name=column
    ))

# Update layout for better visualization
fig.update_layout(
    barmode='stack',
    xaxis_title='Date',
    yaxis_title='Energy (MWh)',
    title='Monthly Energy Flows and Curtailment',
    legend_title='Components',
    xaxis=dict(tickformat="%Y-%m"),
    hovermode='x unified'
)

# Show the interactive chart
fig.show()

In [ ]:
data = pd.concat([NETWORK.links_t.p0[['Solar battery link', 'Solar H2 link', 'Solar load link']], solar_curtailment]).resample('ME').sum()

# Prepare data for Plotly
data = data.reset_index()

# Create an interactive bar chart using Plotly
fig = go.Figure()

for column in data.columns[1:]:  # Skip the first column as it is the index
    fig.add_trace(go.Bar(
        x=data['snapshot'],  # Use the reset index for the x-axis
        y=data[column],
        name=column
    ))

# Update layout for better visualization
fig.update_layout(
    barmode='stack',
    xaxis_title='Date',
    yaxis_title='Energy (MWh)',
    title='Monthly Energy Flows and Curtailment',
    legend_title='Components',
    xaxis=dict(tickformat="%Y-%m"),
    hovermode='x unified'
)

# Show the interactive chart
fig.show()

curtailment = NETWORK.generators_t.p_max_pu[['Onwind park']] * NETWORK.generators.loc['Onwind park']['p_nom_opt'] - NETWORK.generators_t.p[['Onwind park']]
curtailment['Curtailment'] = curtailment['Onwind park']
curtailment.drop(columns=['Onwind park'], inplace=True)

fig1, ax1 = plt.subplots(figsize=(24, 6))
pd.concat([NETWORK.links_t.p0[['Onwind battery link', 'Onwind H2 link', 'Onwind load link']], curtailment]).resample('ME').sum().plot(ax=ax1, kind='bar', stacked=True, legend=True, alpha=0.2)

In [ ]:
NETWORK.loads_t.p[['Desired load']].sum()*3

In [ ]:
curtailment = NETWORK.generators_t.p_max_pu[['Solar park']] * NETWORK.generators.loc['Solar park']['p_nom_opt'] - NETWORK.generators_t.p[['Solar park']]

curtailment.plot(figsize=(24, 6), alpha=0.2)


In [ ]:
fig, ax = plt.subplots(figsize=(24,8))
network.generators_t.p[['Solar park']].abs().plot(ax=ax, kind='area', stacked=True, legend=True, alpha=0.2, title='Power output')
#network.generators_t.p.abs().rolling(window=window_size, center=True).mean().plot(ax=ax, kind='area', stacked=True)

In [ ]:
fig, ax = plt.subplots(figsize=(24,8))
network.stores_t.e[['Battery storage']].abs().plot(ax=ax, kind='area', stacked=True, legend=True, alpha=0.2, title='Power output')

In [8]:
import xarray as xr

renewables_path = paths.input_path / 'renewables'

geo = "14-1486"

capfac_solar = xr.open_dataarray(renewables_path / f"capacity-factor-{geo}-2023-01-2023-12-solar.nc").values.flatten()
capfac_onwind = xr.open_dataarray(renewables_path / f"capacity-factor-{geo}-2023-01-2023-12-onwind.nc").values.flatten()
capfac_offwind = xr.open_dataarray(renewables_path / f"capacity-factor-{geo}-2023-01-2023-12-offwind.nc").values.flatten()

print(f"solar: {capfac_solar.sum()}, onwind: {capfac_onwind.sum()}, offwind: {capfac_offwind.sum()}")

solar: 396.98000979965457, onwind: 1148.573661934868, offwind: 1293.8288220807303
